In [22]:
import warnings
import numpy as np
import matplotlib.pyplot as plt
import requests
from PIL import Image
from io import BytesIO
from lang_sam import LangSAM
import os

In [29]:
def find_coin_masks(image):
    # Suppress warning messages
    warnings.filterwarnings("ignore")
    text_prompt = "coin"
    try:
        model = LangSAM()
        masks, boxes, phrases, logits = model.predict(image, text_prompt)

        if len(masks) == 0:
            print(f"No objects of the '{text_prompt}' prompt detected in the image.")
        else:
            # Convert masks to numpy arrays
            masks_np = [mask.squeeze().cpu().numpy() for mask in masks]
            return masks_np

    except (requests.exceptions.RequestException, IOError) as e:
        print(f"Error: {e}")

# returns y_min, y_max, x_min, x_max
def find_boundary_of_coin(nonzero_indices):
    return nonzero_indices[0,:].min(), nonzero_indices[0,:].max(), nonzero_indices[1,:].min(), nonzero_indices[1,:].max()

def generate_coin_images(image_dir):
    image = Image.open(image_dir).convert("RGB")
    masks = find_coin_masks(image)
    image = np.array(image)
    coins = []
    for index in range(len(masks)):
        mask = np.broadcast_to(np.expand_dims(masks[index],-1), image.shape)
        masked_image = mask * image
        nonzero_indices = np.nonzero(masked_image[:,:,0])
        nonzero_indices = np.array(nonzero_indices)
        y_min, y_max, x_min, x_max = find_boundary_of_coin(nonzero_indices)
        masked_image = masked_image[y_min:y_max,x_min:x_max]
        if (y_max - y_min)<500 and (x_max - x_min)<500:
            difference_y = 500 - (y_max - y_min)
            difference_x = 500 - (x_max - x_min)
            if difference_y != 0:
                if difference_y % 2 == 0:
                    masked_image = np.pad(masked_image, [(difference_y//2, difference_y//2), (0, 0), (0, 0)])
                else:
                    masked_image = np.pad(masked_image, [((difference_y-1)//2, (difference_y-1)//2 + 1), (0, 0), (0, 0)])
            if difference_x != 0:
                if difference_x % 2 == 0:
                    masked_image = np.pad(masked_image, [(0, 0), (difference_x//2, difference_x//2), (0, 0)])
                else:
                    masked_image = np.pad(masked_image, [(0, 0), ((difference_x-1)//2, (difference_x-1)//2 + 1), (0, 0)])
            coins.append(masked_image)
    return coins

def find_last_index(strings):
    max_index = 0
    for string in strings: 
        if string != '.DS_Store':
            index = int(string.replace('img_','').replace('.png',''))
            if index > max_index:
                max_index = index
    return max_index



In [ ]:

image = Image.open("coin_images/10c/IMG_4187.JPG").convert("RGB")
masks = find_coin_masks(image)

# for i in range(len(masks)):
#     mask = masks[i]
#     mask_img = Image.fromarray((mask*255).astype(np.uint8))
#     mask_img.save(f"test_masks/mask_{i}.png")

image = np.array(image)
sizes_y = []
sizes_x = []
for index in range(len(masks)):
        mask = np.broadcast_to(np.expand_dims(masks[index],-1), image.shape)
        masked_image = mask * image
        nonzero_indices = np.nonzero(masked_image[:,:,0])
        nonzero_indices = np.array(nonzero_indices)
        y_min, y_max, x_min, x_max = find_boundary_of_coin(nonzero_indices)
        masked_image = masked_image[y_min:y_max,x_min:x_max]
        sizes_y.append(y_max - y_min)
        sizes_x.append(x_max - x_min)

In [30]:
output_dir = "padded_coin_images/"
dataset_dir = "coin_images/"
subfolders = os.listdir(dataset_dir) # find subfolder names in a folder
if '.DS_Store' in subfolders:
    subfolders.remove('.DS_Store')
print(subfolders)
for subfolder in subfolders:
    files = os.listdir(os.path.join(dataset_dir,subfolder)) # find file names in a folder
    if '.DS_Store' in files:
        files.remove('.DS_Store')
    print(files)
    for file in files:
        padded_coins = generate_coin_images(os.path.join(dataset_dir,subfolder,file))
        print(dataset_dir)
        print(subfolder)
        print(file)
        for padded_coin in padded_coins:
            image = Image.fromarray((padded_coin).astype(np.uint8))
            if os._exists(os.path.join(output_dir, subfolder, '.DS_Store')):
                os.remove(os.path.join(output_dir, subfolder, '.DS_Store'))
            last_index = find_last_index(os.listdir(os.path.join(output_dir, subfolder)))
            image_name = f"img_{last_index+1}.png"
            subfolder_for_padded_coins = os.path.join(output_dir, subfolder, image_name)
            image.save(subfolder_for_padded_coins)


['10c', '50c', '20c', '5c', '1e', '2e']
['IMG_4207.JPG', 'IMG_4191.JPG', 'IMG_4192.JPG', 'IMG_4187.JPG', 'IMG_4208.JPG']
final text_encoder_type: bert-base-uncased


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Model loaded from /Users/gamze/.cache/huggingface/hub/models--ShilongLiu--GroundingDINO/snapshots/a94c9b567a2a374598f05c584e96798a170c56fb/groundingdino_swinb_cogcoor.pth 
 => _IncompatibleKeys(missing_keys=[], unexpected_keys=['label_enc.weight'])
coin_images/
10c
IMG_4207.JPG
final text_encoder_type: bert-base-uncased


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Model loaded from /Users/gamze/.cache/huggingface/hub/models--ShilongLiu--GroundingDINO/snapshots/a94c9b567a2a374598f05c584e96798a170c56fb/groundingdino_swinb_cogcoor.pth 
 => _IncompatibleKeys(missing_keys=[], unexpected_keys=['label_enc.weight'])
coin_images/
10c
IMG_4191.JPG
final text_encoder_type: bert-base-uncased


...

Model loaded from /Users/gamze/.cache/huggingface/hub/models--ShilongLiu--GroundingDINO/snapshots/a94c9b567a2a374598f05c584e96798a170c56fb/groundingdino_swinb_cogcoor.pth 
 => _IncompatibleKeys(missing_keys=[], unexpected_keys=['label_enc.weight'])
coin_images/
2e
IMG_4196.JPG
final text_encoder_type: bert-base-uncased


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Model loaded from /Users/gamze/.cache/huggingface/hub/models--ShilongLiu--GroundingDINO/snapshots/a94c9b567a2a374598f05c584e96798a170c56fb/groundingdino_swinb_cogcoor.pth 
 => _IncompatibleKeys(missing_keys=[], unexpected_keys=['label_enc.weight'])
coin_images/
2e
IMG_4194.JPG
final text_encoder_type: bert-base-uncased


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Model loaded from /Users/gamze/.cache/huggingface/hub/models--ShilongLiu--GroundingDINO/snapshots/a94c9b567a2a374598f05c584e96798a170c56fb/groundingdino_swinb_cogcoor.pth 
 => _IncompatibleKeys(missing_keys=[], unexpected_keys=['label_enc.weight'])
coin_images/
2e
IMG_4195.JPG
